# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

C:\Users\walsa\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\walsa\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\walsa\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df=pd.read_csv('train.csv')  # Load training dataset 
df.head()# Display loaded dataset


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.shape

(891, 12)

In [4]:
df.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [5]:
df.isnull().sum().sum()

866

In [6]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
feature_columns=['Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
X=df[feature_columns]
print(X.head())
y=df[['Survived']]
print(y.head())

   Pclass                                               Name     Sex   Age  \
0       3                            Braund, Mr. Owen Harris    male  22.0   
1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2       3                             Heikkinen, Miss. Laina  female  26.0   
3       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4       3                           Allen, Mr. William Henry    male  35.0   

   SibSp  Parch            Ticket     Fare Cabin Embarked  
0      1      0         A/5 21171   7.2500   NaN        S  
1      1      0          PC 17599  71.2833   C85        C  
2      0      0  STON/O2. 3101282   7.9250   NaN        S  
3      1      0            113803  53.1000  C123        S  
4      0      0            373450   8.0500   NaN        S  
   Survived
0         0
1         1
2         1
3         1
4         0


In [8]:
X_num=X[['Age','SibSp','Parch','Fare']]  # Define numerical columns
X_cat=X[['Pclass','Sex','Embarked']]   # Define categorical colums


In [9]:
# For complete preprocessing of data:
from sklearn.pipeline import Pipeline  # Import Pipeline
from sklearn.pipeline import FeatureUnion  #Import FeatureUnion
from sklearn.preprocessing import MinMaxScaler  #Import MinMax Scaler
from sklearn.preprocessing import OneHotEncoder  # Import OneHotEncoder 
from sklearn.impute import SimpleImputer

In [10]:
# Create list with columns names of numerical attributes
num_attribs = list (X_num)
# Create list with columns names of categorical attributes
cat_attribs = list (X_cat)

In [11]:
# Define class for selection of columns from dataframes
from sklearn.base import BaseEstimator, TransformerMixin
class DFSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [12]:
num_pipeline = Pipeline([
('selector', DFSelector(num_attribs)),
('imputer_num',SimpleImputer(missing_values=np.nan, strategy='mean')),
('std_scaler', MinMaxScaler()),
])
# Define pipeline for categorical features
cat_pipeline = Pipeline([
('selector', DFSelector(cat_attribs)),
('imputer_cat',SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
('one_hot_encoder', OneHotEncoder()),
])

In [13]:
# Merge both pipelines in one
full_pipeline = FeatureUnion(transformer_list=[
("num_pipeline", num_pipeline),
("cat_pipeline", cat_pipeline),
])

X = full_pipeline.fit_transform(X)  # Apply full pipeline on features dataset
X=X.toarray()   # Switch new dataset to Numpy array type



In [14]:
#Shuffle and split prepared datasets to train and test (33%) datasets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

## svm

In [15]:
from sklearn import svm
clf_svm = svm.SVC()
clf_svm.fit(X_train, np.ravel(y_train))
acc_svm = clf_svm.score(X_test, y_test)
print(acc_svm)

0.8406779661016949


## KNeighborsClassifier

In [16]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn =KNeighborsClassifier()
clf_knn.fit(X_train,  np.ravel(y_train))

acc_knn = clf_knn.score(X_test, y_test)
print(acc_knn)

0.8135593220338984


## PCA with NB

In [23]:
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
pca = PCA(n_components=8)
X_train1 = pca.fit_transform(X_train)
X_test1 = pca.transform(X_test)


gnb = GaussianNB()
gnb.fit(X_train1, y_train)
acc_pca = gnb.score(X_test1, y_test)#/100
print(acc_pca)

0.8033898305084746


C:\Users\walsa\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


## RidgeClassifier

In [24]:
from sklearn.linear_model import RidgeClassifier

clf_ridge =RidgeClassifier()
clf_ridge.fit(X_train,  np.ravel(y_train))

acc_ridge = clf_ridge.score(X_test, y_test)
print(acc_ridge)

0.8203389830508474


## AdaBoostClassifier

In [25]:
from sklearn.ensemble import AdaBoostClassifier

clf_AdaBoost =AdaBoostClassifier()
clf_AdaBoost.fit(X_train,  np.ravel(y_train))

acc_AdaBoost = clf_AdaBoost.score(X_test, y_test)
print(acc_AdaBoost)

0.8372881355932204


## Neural Network

In [26]:
from sklearn.neural_network import MLPClassifier
clf_nn=MLPClassifier()
clf_nn.fit(X_train,  np.ravel(y_train))
acc_nn = clf_nn.score(X_test, y_test)
print(acc_nn)

0.8372881355932204


C:\Users\walsa\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Quadratic Discriminant Analysis

In [27]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
clf_qda=QuadraticDiscriminantAnalysis()
clf_qda.fit(X_train,  np.ravel(y_train))

acc_qda = clf_qda.score(X_test, y_test)
print(acc_qda)

0.8135593220338984


C:\Users\walsa\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


## Linear Discriminant Analysis

In [28]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf_lda=LinearDiscriminantAnalysis()
clf_lda.fit(X_train,  np.ravel(y_train))

acc_lda = clf_lda.score(X_test, y_test)
print(acc_lda)

0.8203389830508474


## VotingClassifier

In [29]:
from sklearn.ensemble import VotingClassifier
clf_voting = VotingClassifier(estimators=[('qda', clf_qda), ('svm', clf_svm), ('knn', clf_knn)], voting='hard')
clf_voting.fit(X_train,  np.ravel(y_train))

acc_voting = clf_voting.score(X_test, y_test)
print(acc_voting)


0.8406779661016949


C:\Users\walsa\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


## Passive Aggressive Classifier

In [30]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf_pac=PassiveAggressiveClassifier()
clf_pac.fit(X_train,  np.ravel(y_train))

acc_pac= clf_pac.score(X_test, y_test)
print(acc_pac)

0.7898305084745763


# Evaluation stage

In [31]:
#f1 score
from sklearn.metrics import f1_score
print(f1_score(y_test,clf_svm.predict(X_test), average=None))
print(f1_score(y_test,clf_knn.predict(X_test), average=None))
print(f1_score(y_test,clf_pca.predict(X_test), average=None))
print(f1_score(y_test,clf_ridge.predict(X_test), average=None))
print(f1_score(y_test,clf_AdaBoost.predict(X_test), average=None))
print(f1_score(y_test,clf_nn.predict(X_test), average=None))
print(f1_score(y_test,clf_qda.predict(X_test), average=None))
print(f1_score(y_test,clf_lda.predict(X_test), average=None))
print(f1_score(y_test,clf_voting.predict(X_test), average=None))
print(f1_score(y_test,clf_pac.predict(X_test), average=None))

[0.88220551 0.7539267 ]
[0.85411141 0.74178404]
[0.85714286 0.75799087]
[0.86956522 0.78378378]
[0.87564767 0.76470588]
[0.86075949 0.71794872]
[0.85714286 0.75799087]
[0.88101266 0.75897436]
[0.8258427  0.73504274]
